In [1]:

!pip install mlflow boto3 awscli

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: C:\Users\DARSHAN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Load AWS credentials from .env file
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(dotenv_path='../.env')

# Set AWS credentials as environment variables
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')
os.environ['AWS_DEFAULT_REGION'] = os.getenv('AWS_DEFAULT_REGION', 'us-east-1')

print("AWS credentials loaded successfully!")
print(f"Region: {os.environ.get('AWS_DEFAULT_REGION')}")

AWS credentials loaded successfully!
Region: us-east-1


In [3]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-184-72-133-224.compute-1.amazonaws.com:5000/")

C:\Users\DARSHAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

mlflow.set_experiment("Exp 3 - TfIdf Trigram max_features")
     


2026/02/11 00:04:36 INFO mlflow.tracking.fluent: Experiment with name 'Exp 3 - TfIdf Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-darshan/3', creation_time=1770748476502, experiment_id='3', last_update_time=1770748476502, lifecycle_stage='active', name='Exp 3 - TfIdf Trigram max_features', tags={}>

In [5]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [6]:
df = pd.read_csv(r'E:\PROJECT_3rd\end-to-end MLops\notebook\reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [7]:
# Step 1: Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)  # Trigram setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2026/02/11 00:06:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
C:\Users\DARSHAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run TFIDF_Trigrams_max_features_1000 at: http://ec2-184-72-133-224.compute-1.amazonaws.com:5000/#/experiments/3/runs/157b9cbf4dc04e3f8d14cb4537ef9ef1
🧪 View experiment at: http://ec2-184-72-133-224.compute-1.amazonaws.com:5000/#/experiments/3


2026/02/11 00:08:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
C:\Users\DARSHAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run TFIDF_Trigrams_max_features_2000 at: http://ec2-184-72-133-224.compute-1.amazonaws.com:5000/#/experiments/3/runs/9e5857cb9d7f4e519e550332bfa5f578
🧪 View experiment at: http://ec2-184-72-133-224.compute-1.amazonaws.com:5000/#/experiments/3


2026/02/11 00:11:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
C:\Users\DARSHAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


...

🏃 View run TFIDF_Trigrams_max_features_8000 at: http://ec2-184-72-133-224.compute-1.amazonaws.com:5000/#/experiments/3/runs/867414387f5c49809c1a2dbfce10eeac
🧪 View experiment at: http://ec2-184-72-133-224.compute-1.amazonaws.com:5000/#/experiments/3


2026/02/11 00:22:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
C:\Users\DARSHAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run TFIDF_Trigrams_max_features_9000 at: http://ec2-184-72-133-224.compute-1.amazonaws.com:5000/#/experiments/3/runs/e34a80b61a8942a889ca0751465519fa
🧪 View experiment at: http://ec2-184-72-133-224.compute-1.amazonaws.com:5000/#/experiments/3


2026/02/11 00:25:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
C:\Users\DARSHAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


🏃 View run TFIDF_Trigrams_max_features_10000 at: http://ec2-184-72-133-224.compute-1.amazonaws.com:5000/#/experiments/3/runs/1fbe46e9acc04326824972100ca5da07
🧪 View experiment at: http://ec2-184-72-133-224.compute-1.amazonaws.com:5000/#/experiments/3
